In [1]:
!pip install boto3 pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 10.4 MB/s eta 0:00:00


In [2]:
!curl https://idp-assets-wwso.s3.us-east-2.amazonaws.com/workshop-data/docClassificationSamples.zip --output /content/docClassificationSamples.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 56.9M  100 56.9M    0     0  76.9M      0 --:--:-- --:--:-- --:--:-- 76.9M


In [3]:
!unzip /content/docClassificationSamples.zip -d /content

Archive:  /content/docClassificationSamples.zip
   creating: /content/sampleGallery/
   creating: /content/sampleGallery/BankStatements/
  inflating: /content/sampleGallery/BankStatements/PacificWesternBankStatement__page1__pg100087__127.png  
 extracting: /content/sampleGallery/BankStatements/ValleyBankStatement__page1__pg1000126__031.png  
 extracting: /content/sampleGallery/BankStatements/WebsterBankStatement__page1__pg1000136__042.png  
   creating: /content/sampleGallery/ClosingDisclosure/
  inflating: /content/sampleGallery/ClosingDisclosure/ClosingDisclosure_page1_1.jpeg  
  inflating: /content/sampleGallery/ClosingDisclosure/ClosingDisclosure_page1_2.jpeg  
   creating: /content/sampleGallery/Invoices/
  inflating: /content/sampleGallery/Invoices/invoice1.tif  
  inflating: /content/sampleGallery/Invoices/invoice2.png  
  inflating: /content/sampleGallery/Invoices/invoice3.tif  
   creating: /content/sampleGallery/SSCards/
  inflating: /content/sampleGallery/SSCards/SSC__pagex_

In [4]:
import json
import boto3
import base64
import os

bedrock_runtime=boto3.client('bedrock-runtime',region_name='us-east-1',aws_access_key_id='{Put your AWS Access Key here}',aws_secret_access_key='{Put your AWS Secret Access Key here}')

from pinecone import Pinecone

pc = Pinecone(api_key="{Put your Pinecone API Key here}")
index = pc.Index("{Put your Pinecone Index name here}")

from io import BytesIO
from PIL import Image

MAX_IMAGE_HEIGHT: int = 2048
MAX_IMAGE_WIDTH: int = 2048

def resizeandGetByteData(imageFile):
    image = Image.open(imageFile)
    if (image.size[0] * image.size[1]) > (MAX_IMAGE_HEIGHT * MAX_IMAGE_WIDTH):
        image = image.resize((MAX_IMAGE_HEIGHT, MAX_IMAGE_WIDTH))
    with BytesIO() as output:
        image.save(output, 'png')
        bytes_data = output.getvalue()
    return bytes_data


def image_embedding(image_name):
  bytes_data = resizeandGetByteData(image_name)
  input_image = base64.b64encode(bytes_data).decode('utf8')
  body = json.dumps(
    {
        "inputImage": input_image
    }
  )
  response = bedrock_runtime.invoke_model(
  body=body,
  modelId="amazon.titan-embed-image-v1",
  accept="application/json",
  contentType="application/json"
  )
  response_body = json.loads(response.get("body").read())
  return response_body.get("embedding")

def get_image_files_from_directory(directory):
    return [os.path.join(directory, file) for file in os.listdir(directory)]

# Directory containing the images of BankStatements
image_directory = '/content/sampleGallery/BankStatements'

# Get list of image files in the BankStatements directory
data_to_store = get_image_files_from_directory(image_directory)

print("Storing BankStatements data : ",data_to_store)

vectors=[]
for i in data_to_store:
  vectors.append({
            "id":i,
            "values": image_embedding(i),
            "metadata": {"category": "BankStatements"}
        })

index.upsert(
    vectors=vectors,
    namespace= "ns1"
)


vectors=[]
# Directory containing the images of ClosingDisclosure
image_directory = '/content/sampleGallery/ClosingDisclosure'

# Get list of image files in the ClosingDisclosure directory
data_to_store = get_image_files_from_directory(image_directory)

print("Storing ClosingDisclosure data : ",data_to_store)

vectors=[]
for i in data_to_store:
  vectors.append({
            "id":i,
            "values": image_embedding(i),
            "metadata": {"category": "ClosingDisclosure"}
        })

index.upsert(
    vectors=vectors,
    namespace= "ns1"
)

# Directory containing the images of Emails
image_directory = '/content/sampleGallery/Emails'

# Get list of image files in the Emails directory
data_to_store = get_image_files_from_directory(image_directory)

print("Storing Emails data : ",data_to_store)

vectors=[]
for i in data_to_store:
  vectors.append({
            "id":i,
            "values": image_embedding(i),
            "metadata": {"category": "Emails"}
        })

index.upsert(
    vectors=vectors,
    namespace= "ns1"
)

# Directory containing the images of Invoices
image_directory = '/content/sampleGallery/Invoices'

# Get list of image files in the Invoices directory
data_to_store = get_image_files_from_directory(image_directory)

print("Storing Invoices data : ",data_to_store)

vectors=[]
for i in data_to_store:
  vectors.append({
            "id":i,
            "values": image_embedding(i),
            "metadata": {"category": "Invoices"}
        })

index.upsert(
    vectors=vectors,
    namespace= "ns1"
)

# Directory containing the images of SSCards
image_directory = '/content/sampleGallery/SSCards'

# Get list of image files in the SSCards directory
data_to_store = get_image_files_from_directory(image_directory)

print("Storing SSCards data : ",data_to_store)

vectors=[]
for i in data_to_store:
  vectors.append({
            "id":i,
            "values": image_embedding(i),
            "metadata": {"category": "SSCards"}
        })

index.upsert(
    vectors=vectors,
    namespace= "ns1"
)

# Directory containing the images of W4
image_directory = '/content/sampleGallery/W4'

# Get list of image files in the W4 directory
data_to_store = get_image_files_from_directory(image_directory)

print("Storing W4 data : ",data_to_store)

vectors=[]
for i in data_to_store:
  vectors.append({
            "id":i,
            "values": image_embedding(i),
            "metadata": {"category": "W4"}
        })

index.upsert(
    vectors=vectors,
    namespace= "ns1"
)

Storing BankStatements data :  ['/content/sampleGallery/BankStatements/PacificWesternBankStatement__page1__pg100087__127.png', '/content/sampleGallery/BankStatements/WebsterBankStatement__page1__pg1000136__042.png', '/content/sampleGallery/BankStatements/ValleyBankStatement__page1__pg1000126__031.png']
Storing ClosingDisclosure data :  ['/content/sampleGallery/ClosingDisclosure/ClosingDisclosure_page1_1.jpeg', '/content/sampleGallery/ClosingDisclosure/ClosingDisclosure_page1_2.jpeg']
Storing Emails data :  ['/content/sampleGallery/Emails/email1.tif']
Storing Invoices data :  ['/content/sampleGallery/Invoices/invoice2.png', '/content/sampleGallery/Invoices/invoice1.tif', '/content/sampleGallery/Invoices/invoice3.tif']
Storing SSCards data :  ['/content/sampleGallery/SSCards/SSC__pagex__SSC1__001.jpg', '/content/sampleGallery/SSCards/SSC__pagex__SSC1__004.jpg']
Storing W4 data :  ['/content/sampleGallery/W4/W4_page1_1.jpg', '/content/sampleGallery/W4/W4_page1_2.jpeg']


{'upserted_count': 2}

In [5]:
classification=index.query(
    namespace="ns1",
    vector=image_embedding('/content/testGallery/Atlantic_BankStatement.png'),
    include_metadata=True,
    top_k=1
)
#print(classification)
classification['matches'][0]['metadata']['category']

'BankStatements'